In [1]:
import lightgbm as lgb
import glob
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('data/train.csv')

In [3]:
def calcualte_acc(x, y):
    count = 0
    for i, j in enumerate(x):
        if j == y[i]:
            count += 1
    print(count/len(x))

In [4]:
mobile_paths = glob.glob('predictions/mobile/*')

In [5]:
mobile_paths

['predictions/mobile\\conv1d_1',
 'predictions/mobile\\conv1d_2layers',
 'predictions/mobile\\conv1d_3',
 'predictions/mobile\\gru',
 'predictions/mobile\\lstm']

In [9]:
test_paths = ['predictions/test/conv1d_1.npy',
'predictions/test/conv1d_2layers.npy',
 'predictions/test/conv1d_3.npy',
 'predictions/test/gru.npy',
 'predictions/test/lstm.npy']

In [10]:
test_paths

['predictions/test/conv1d_1.npy',
 'predictions/test/conv1d_2layers.npy',
 'predictions/test/conv1d_3.npy',
 'predictions/test/gru.npy',
 'predictions/test/lstm.npy']

In [11]:
test_set = np.load(test_paths[0])[:, 31:]
print(test_set.shape)
for i in test_paths[1:]:
    test_set = np.append(test_set, np.load(i)[:, 31:], axis=1)

(172402, 27)


In [12]:
test_set = test_set[131985:, :]

In [13]:
num_classes = 27

In [14]:
params={
        "objective" : "multiclass",
        "num_class" : num_classes,
        "num_leaves" : 63,
        "max_depth": -1,
        "learning_rate" : 0.01,
        "bagging_fraction" : 0.9,  # subsample
        "feature_fraction" : 0.9,  # colsample_bytree
        "bagging_freq" : 5,        # subsample_freq
        "bagging_seed" : 2019,
        'max_bin': 63,
        #'device': 'gpu',
        'metric': 'multi_logloss',
}

In [18]:
for j in range(len(mobile_paths)):
    for i in range(5):
        print(glob.glob(mobile_paths[j]+'/*')[i])
        print(np.load(glob.glob(mobile_paths[j]+'/*')[i]).shape)

predictions/mobile\conv1d_1\mobile_predictions_0.npy
(32066, 27)
predictions/mobile\conv1d_1\mobile_predictions_1.npy
(32066, 27)
predictions/mobile\conv1d_1\mobile_predictions_2.npy
(32066, 27)
predictions/mobile\conv1d_1\mobile_predictions_3.npy
(32066, 27)
predictions/mobile\conv1d_1\mobile_predictions_4.npy
(32066, 27)
predictions/mobile\conv1d_2layers\mobile_predictions_0.npy
(32066, 27)
predictions/mobile\conv1d_2layers\mobile_predictions_1.npy
(32066, 27)
predictions/mobile\conv1d_2layers\mobile_predictions_2.npy
(32066, 27)
predictions/mobile\conv1d_2layers\mobile_predictions_3.npy
(32066, 27)
predictions/mobile\conv1d_2layers\mobile_predictions_4.npy
(32066, 27)
predictions/mobile\conv1d_3\mobile_predictions_0.npy
(32066, 27)
predictions/mobile\conv1d_3\mobile_predictions_1.npy
(32066, 27)
predictions/mobile\conv1d_3\mobile_predictions_2.npy
(32066, 27)
predictions/mobile\conv1d_3\mobile_predictions_3.npy
(32066, 27)
predictions/mobile\conv1d_3\mobile_predictions_4.npy
(32066,

In [ ]:
test_output = np.zeros((test_set.shape[0], num_classes))
for i in range(5):
    print('fold_{}'.format(i))
    lst = [0,1,2,3,4]
    lst.remove(i)

    trn_set = np.load(glob.glob(mobile_paths[0]+'/*')[lst[0]])
    val_set = np.load(glob.glob(mobile_paths[0]+'/*')[i])
    
    for j in mobile_paths[1:]:
        trn_set = np.append(trn_set, np.load(glob.glob(j+'/*')[lst[0]]), axis=1)
        val_set = np.append(val_set, np.load(glob.glob(j+'/*')[i]), axis=1)
    #trn_set = trn_set.reshape(trn_set.shape[0], 1)
        
    for x in lst[1:]:
        one_row = np.load(glob.glob(mobile_paths[0]+'/*')[x])
        for j in mobile_paths[1:]:
            one_row = np.append(one_row, np.load(glob.glob(j+'/*')[x]), axis=1)
        trn_set = np.append(trn_set, one_row, axis=0)
    
    indexes = np.load('np_array/trn_index_m_fold.npy')
    index = int(glob.glob(mobile_paths[0] + '/*')[i][-5])
    print(index)
    val_index = indexes[index]
    other_lst = [0,1,2,3,4]
    other_lst.remove(index)
    trn_index = indexes[other_lst[0]]
    other_lst.remove(other_lst[0])
    for num in other_lst:
        trn_index = np.append(trn_index, indexes[num])
        
    labels_trn = train.iloc[trn_index, 2]
    labels_trn = np.array(labels_trn) - 31

    labels_val = train.iloc[val_index, 2]
    labels_val = np.array(labels_val) - 31
    
    print(trn_set.shape)
    print(val_set.shape)
    lgtrain, lgval = lgb.Dataset(trn_set, labels_trn), lgb.Dataset(val_set, labels_val)
    lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=100)
    pred = lgbmodel.predict(val_set)
    
    predictions = []
    for i in pred:
        predictions.append(list(i).index(i.max()))
    calcualte_acc(predictions, labels_val)
    
    np.save('first_layer/five_nlp_{}_rf'.format(index), pred)
    pred = lgbmodel.predict(test_set)
    test_output += pred/5

fold_0
0
(128264, 135)
(32066, 135)
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 1.10285	valid_1's multi_logloss: 1.18026
[200]	training's multi_logloss: 0.680044	valid_1's multi_logloss: 0.789668
[300]	training's multi_logloss: 0.510354	valid_1's multi_logloss: 0.649814
[400]	training's multi_logloss: 0.424194	valid_1's multi_logloss: 0.593471
[500]	training's multi_logloss: 0.370371	valid_1's multi_logloss: 0.568051
[600]	training's multi_logloss: 0.333826	valid_1's multi_logloss: 0.555902
[700]	training's multi_logloss: 0.307711	valid_1's multi_logloss: 0.550305
[800]	training's multi_logloss: 0.287802	valid_1's multi_logloss: 0.548128
[900]	training's multi_logloss: 0.271406	valid_1's multi_logloss: 0.548073
Early stopping, best iteration is:
[857]	training's multi_logloss: 0.278105	valid_1's multi_logloss: 0.547841
0.8347470841389634
fold_1
1
(128264, 135)
(32066, 135)
Training until validation scores don't improve for 100 rounds.


In [23]:
np.save('first_layer/mobile_five_test', test_output)

In [24]:
test_output.shape

(40417, 27)

In [30]:
calcualte_acc(predictions, labels_val)

0.7924350541724096
